Aim of the notebook is it to convert the webscraped MIDI files representing full songs into snippets which represent the content of the already provided MP3 tracks.

In [1]:
import pandas as pd
import mido
from mido import MidiFile
import re
import numpy as np
import librosa

import sys

import os
import glob
import shutil

In [2]:
sys.path.insert(0, '../../../stage-1/overall_used_tools')
import requirements_check as rc

rc.check(sys, [pd,mido,re,np,librosa,os,glob,shutil])

Your version identical with original version 
-> pandas 
-> 1.3.5


Your version identical with original version 
-> mido 
-> 1.2.10


Your version identical with original version 
-> re 
-> 2.2.1


Your version identical with original version 
-> numpy 
-> 1.21.6


Your version identical with original version 
-> librosa 
-> 0.8.1


For 
os 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


For 
glob 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Possibly different versions: 
-> shutil 
-> original version 1.0.0 
-> your version  is not clearly visible. Go to your used python folder path .../python3.7/site-packages to investigate the version of your package. Also possible: have a look at the shell commands "pip show module_name" and "apt show module_name"


Your version identical with original version 
-> sys 
-> 3.7.3




In [3]:
sys.path.insert(0,'../../../stage2_feature_extraction/symbolicFeatureExtraction') 
import basic_info_extraction as bie

Your version identical with original version 
-> numpy 
-> 1.21.6


Your version identical with original version 
-> pandas 
-> 1.3.5


Your version identical with original version 
-> mido 
-> 1.2.10


Your version identical with original version 
-> sys 
-> 3.7.3




In [5]:
info_df = pd.read_csv('song_overview.csv')
info_df

,code,song_code,artist,title,year,genre,subgenre,tempo (BPM),key,energetics,instrumentation_groups,instrumentation,language,explicit_new,lyrics timeframe,timestamp_start,timestamp_end,duration_sec,url_excerpt
0,H_2Pac_AllEyez,H015,2Pac,All Eyez On Me,1996.0,Hip-Hop/Rap,Gangsta Rap,93,NaN,mittel,instrumental + vocal,NaN,en,yes,One nigga's gettin' jealous and motherfuckers ...,00:01:06,00:01:40,34.0,https://www.youtube.com/embed/05PCmqjIeNE?star...
1,H_2Pac_KeepYaH,H060,2Pac,Keep Ya Head Up,1993.0,Hip-Hop/Rap,Gangsta Rap,86,NaN,niedrig-mittel,instrumental + vocal,NaN,en,NaN,"Keep ya head up, ooh, child, things are gonna ...",00:01:22,00:02:03,41.0,https://www.youtube.com/embed/sPS54-fIpVs?star...
2,H_50Cent_CandySh,H059,50 Cent ft. Olivia,Candy Shop,2005.0,Hip-Hop/Rap,gangsta Rap,98,NaN,mittel-schnell,instrumental + vocal,NaN,en,NaN,Welcome to the candy shop. Yeah. Uh-huh. So se...,00:00:25,00:01:13,48.0,NaN
3,H_50Cent_InDaClu,H041,50 Cent,In Da Club,2003.0,Hip-Hop/Rap,Gangsta Rap,90,NaN,mittel-hoch,instrumental + vocal,NaN,en,yes,"Go, go, go, go, go, go, Go, shorty. It's your ...",00:00:40,00:01:18,38.0,https://www.youtube.com/embed/5qm8PH4xAss?star...
4,H_ActionBronson_ActinCr,H047,Action Bronson,Actin Crazy,2015.0,Hip-Hop/Rap,Hip-Hop/Rap,166 Halftime,NaN,mittel-hoch,instrumental + vocal,NaN,en,yes,"Ma, you know I'm still your little baby. All m...",00:00:15,00:01:12,57.0,https://www.youtube.com/embed/debIyWS6Byc?star...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,P_VanessaCarlton_AThousa,P067,Vanessa Carlton,A Thousand Miles,2001.0,Pop,NaN,95,NaN,mittel,instrumental + vocal,NaN,en,NaN,Staring blankly ahead \n Just making my way \n...,00:00:40,00:01:26,46.0,https://www.youtube.com/embed/Cwkej79U3ek?star...
366,P_Weeknd_Starboy,P026,The Weeknd,Starboy,2016.0,Pop,R&B,93,NaN,mittel-hoch,instrumental + vocal,NaN,en,NaN,"Main bitch outta your league too, ah \n Side b...",00:01:12,00:01:43,31.0,https://www.youtube.com/embed/34Na4j8AVgA?star...
367,P_WhitneyHouston_IWannaD,P057,Whitney Houston,I Wanna Dance With Somebody,1987.0,Pop,R&B,119,C#Major,hoch,instrumental + vocal,NaN,en,NaN,Still enough time to figure out \n How to chas...,00:01:06,00:01:44,38.0,https://www.youtube.com/embed/eH3giaIzONA?star...
368,P_Wrabel_TheVill,P083,Wrabel,The Village,2017.0,Pop,Acoustic,122,NaN,NaN,instrumental + vocal,NaN,en,NaN,"Tell you, ""Wake up, go put on your makeup"" \n ...",00:01:28,00:02:13,45.0,https://www.youtube.com/embed/tilsrO-3gcQ?star...


In [6]:
time_start = info_df['timestamp_start']
time_end = info_df['timestamp_end']
duration = info_df['duration_sec']
mp3_code = info_df['code']
tempo = info_df['tempo (BPM)']
url = info_df['url_excerpt']
title =  info_df['title']
artist =  info_df['artist']

In [8]:
# missing durations are filled up by looking at the beloning audio files:
ind_counter = 0
missing_time_stamps = []
for c,ts,d in zip(mp3_code,time_start,duration):
    
    if np.isnan(d):
        print('No duration information for: ' + c)
        y, sr = librosa.load('../../audio2midi_converter/track_preparation/full_wav_generated/' + c + '.wav')
        duration_lib = librosa.get_duration(y=y, sr=sr) # in seconds
        info_df.at[ind_counter, 'duration_sec'] = duration_lib
    try:
        if np.isnan(str(ts)):
            print('No time start info for: ' + c)
            missing_time_stamps.append(c)
    except:
        pass
    ind_counter += 1
    
# Manually listening into the songs with missing time information to fix the start time of the snippet
# give us:
start_time_dict = {'H_AedFX_Warning':'00:00:29' ,'H_ClaptuClows_sadocto':'00:00:15','H_DECAP_Panther':'00:00:11','H_Dexter_Everyda':'00:00:12','H_DjShadow_Buildin':'00:00:50','H_JoeyPecoraro_YourFav':'00:01:32','H_Philanthropextusken_pueblo':'00:00:23','H_quicklyquickly_swingth':'00:00:40','H_TomDoolie_BlueBoo':'00:00:18','K_Bartok_PianoSo':'00:09:20','K_Bruch_KolNidr':'00:00:40','K_Milhaud_Scaramo':'00:00:09','K_Chopin_Concert':'00:02:25','K_Delibes_Coppeli':'00:01:39','K_Holst_ThePlan':'00:00:52','K_Mahler_6Sympho':'00:03:15'}

# Put found start times into dataframe:
for key, value in start_time_dict.items():
    curr_ind = info_df[info_df['code']==key].index[0]
    info_df.at[curr_ind, 'timestamp_start'] = value

# samples with missing time info but starting time of 0 getting the time update in the dataframe:
info_df['timestamp_start'] = info_df['timestamp_start'].fillna('00:00:00')



No duration information for: H_AedFX_Warning
No duration information for: H_brillion_Aurora
No duration information for: H_Claptu_Aper
No duration information for: H_ClaptuClows_sadocto
No duration information for: H_DECAP_Panther
No duration information for: H_Deli_5_32
No duration information for: H_Dexter_Everyda
No duration information for: H_DjShadow_Buildin
No duration information for: H_FigubBrazlevic_SilentL
No duration information for: H_FloFilz_keepitr
No duration information for: H_idealism_control
No duration information for: H_JoeyPecoraro_YourFav
No duration information for: H_LordFinesse_MidasEr
No duration information for: H_montebooker_23
No duration information for: H_NoSpirit_Glowing
No duration information for: H_Philanthropextusken_pueblo
No duration information for: H_quicklyquickly_swingth
No duration information for: H_SuffDaddy_KCVodka
No duration information for: H_TomDoolie_BlueBoo
No duration information for: H_tomppabeats_onetwos
No duration information for

In [13]:
# find out the tempo for each sample and bring it down to bps/beats per second:

bps_list = [] # beats per second
        
for single_tempo in tempo.values:
    try:
        bpm = float(single_tempo)
    except:
        if 'halftime' in single_tempo.lower():
            bpm = float(re.search(r'\d+',single_tempo).group()) / 2
            #  Half-time means playing a rhythm half as fast as at what it was previously played.
            # https://www.linkedin.com/learning/music-theory-for-songwriters-rhythm/double-time-and-half-time
        
        else: # collect the numbers you can find

            re_list = re.split(r'(\d+)', single_tempo) # split the elements in a string where the numbers are

            val_list = []
            for ele in re_list:
                try:
                    # avoid getting out tact which we dont need:
                    if ind>0:
                        ele_before = re_list[ind-1].replace(' ','')
                    if ind<len(re_list)-1: # before last index
                        ele_after = re_list[ind+1].replace(' ','')
                    if not (ele_before=='(' and ele_after=='/') and not (ele_before=='/' and ele_after==')'):
                        # when no tact the case then we can append our list with found tempo number:
                        val_list.append(float(ele))

                except:
                    pass

            if len(val_list)>1:
                bpm = np.array(val_list).mean()
            if len(val_list)==1: 
                bpm = val_list[0]
            else:
                # simply using the speed of midi in the assumption that midi creator took circa the 
                # correct tempo, later
                bpm = -10

    
    bps_list.append(bpm/60)
    # where bps will is smaller than 0 replace later with midi tempo
    
info_df['BPS'] = bps_list



In [16]:
# split the time in its components:
info_df[['hour', 'minute', 'second']] = info_df['timestamp_start'].str.split(':', 3, expand=True)

# compute start time snippet in seconds only:
min2sec = info_df['minute'].values.astype(int)*60
hour2sec = info_df['hour'].values.astype(int)*60*60
total_sec_start = info_df['second'].values.astype(float) + min2sec + hour2sec

info_df['total_sec_start'] = total_sec_start

# compute when mp3 snippet ends:
durations = info_df['duration_sec'].values.astype(float)

info_df['total_sec_end'] = total_sec_start + durations

# save the updated dataframe:
info_df.to_csv('song_overview_updated.csv', index=False)

In [17]:
info_df = pd.read_csv('song_overview_updated.csv')

In [18]:
info_df.head()

,code,song_code,artist,title,year,genre,subgenre,tempo (BPM),key,energetics,...,timestamp_start,timestamp_end,duration_sec,url_excerpt,BPS,hour,minute,second,total_sec_start,total_sec_end
0,H_2Pac_AllEyez,H015,2Pac,All Eyez On Me,1996.0,Hip-Hop/Rap,Gangsta Rap,93,NaN,mittel,...,00:01:06,00:01:40,34.0,https://www.youtube.com/embed/05PCmqjIeNE?star...,1.550000,0,1,6,66.0,100.0
1,H_2Pac_KeepYaH,H060,2Pac,Keep Ya Head Up,1993.0,Hip-Hop/Rap,Gangsta Rap,86,NaN,niedrig-mittel,...,00:01:22,00:02:03,41.0,https://www.youtube.com/embed/sPS54-fIpVs?star...,1.433333,0,1,22,82.0,123.0
2,H_50Cent_CandySh,H059,50 Cent ft. Olivia,Candy Shop,2005.0,Hip-Hop/Rap,gangsta Rap,98,NaN,mittel-schnell,...,00:00:25,00:01:13,48.0,NaN,1.633333,0,0,25,25.0,73.0
3,H_50Cent_InDaClu,H041,50 Cent,In Da Club,2003.0,Hip-Hop/Rap,Gangsta Rap,90,NaN,mittel-hoch,...,00:00:40,00:01:18,38.0,https://www.youtube.com/embed/5qm8PH4xAss?star...,1.500000,0,0,40,40.0,78.0
4,H_ActionBronson_ActinCr,H047,Action Bronson,Actin Crazy,2015.0,Hip-Hop/Rap,Hip-Hop/Rap,166 Halftime,NaN,mittel-hoch,...,00:00:15,00:01:12,57.0,https://www.youtube.com/embed/debIyWS6Byc?star...,1.383333,0,0,15,15.0,72.0


### Here comes MIDI into the game
We have as example: <br>
1 BPS_audio = 1 quarter_note/sec : 40 quarter_notes needed when snippet of sample start at sec 40<br>
2 BPS_midi                        : x<br>
x = BPS_midi * 40 quarter_notes / BPS_audio --> gives us the number of needed quarter notes<br>
midi duration notation of quarter_note: 48 --> for our example time to pass: 40*48=1920

In [23]:
def tempo_timeframe_computer(mid, info_df, mp3_code):

    # compute midi tempo:
    # aim: get the first track setting the tempo at the earliest time moment
    found = False
    tempo_midi = 0
    store_moment = -1
    stored_track = -1
    for ind,track in enumerate(mid.tracks): # first track setting tempo is the leading track in setting tempo
        for msg in track:
            if msg.type == 'set_tempo':

                if store_moment == -1 or (store_moment == msg.time and stored_track == ind) or (msg.time < store_moment and stored_track != ind):
                    store_moment = msg.time
                    tempo_midi = msg.tempo
                    stored_track = ind
                    found = True

                elif store_moment < msg.time: 
                    # of the first time moment with setting tempo we already achieved the last one
                    break


    bpm_midi =  60000000/tempo_midi # or use mido.tempo2bpm(tempo_midi)
    bps_midi = bpm_midi/60

    # compute the needed time snippet - the timeframe:
    curr_line = info_df[info_df['code']==mp3_code]
    
    if len(curr_line) == 0: # the searched sample is not in the collection 
        
        # find out duration of midi file:
        if mid.type < 2:
            durat_sec_mid = mid.length

        else: # type 2 tracks are asynchronous. We have several tracks of different length. Find the longest one  
            durat_sec_mid = 0
            for track in mid.tracks:

                tempo_midi_track = 550458 # simply assume it
                track_duration = 0
                curr_tick = 0

                for msg in track:

                    if msg.type == 'set_tempo':

                        tempo_midi_track = msg.tempo

                    elif msg.type == 'note_on' or msg.type == 'note_off':

                        curr_sec = mido.tick2second(msg.time, mid.ticks_per_beat, tempo=tempo_midi_track)
                        track_duration += curr_sec

                durat_sec_mid = max(track_duration, durat_sec_mid)
                
        # take the mean of the midi file:
        if durat_sec_mid <= 30:
            return -1, -1 # then the whole file will be stored

        else:
            middle_piece = durat_sec_mid/2
            start_sec_collection = middle_piece - 15
            end_sec_collection = middle_piece + 15
            
            
            if found == False: # no tempo in midi found, collection has also no tempo/current sample
                bps_midi = 100  

            tempo_bps_collection = bps_midi
            tempo_midi = mido.bpm2tempo(bps_midi*60) 
 
                
    else:    
        start_sec_collection = curr_line['total_sec_start'].iloc[0] # iloc[0] to get first value
        end_sec_collection = curr_line['total_sec_end'].iloc[0]
        tempo_bps_collection = curr_line['BPS'].iloc[0]
   
    
        if found == False: # for the case that in midi file no tempo set choose the tempo from the data collection 
            # as replacement 
            bps_midi = tempo_bps_collection
            tempo_midi = mido.bpm2tempo(bps_midi*60) 

        elif np.isnan(tempo_bps_collection) and np.isnan(bps_midi): # both tempi are note given
            bps_midi = 100 # not important which value here is given because below, computing start_midi and 
            # end_midi they get cancelled out.
            tempo_bps_collection = bps_midi
            tempo_midi = mido.bpm2tempo(bps_midi*60) 

        elif np.isnan(tempo_bps_collection) or tempo_bps_collection<0:
            tempo_bps_collection = bps_midi

        
    # we want to cut out the relevant time frame within the midi file. We need the start and end notation:
    seconds_to_aim_per_midi_tempo = ((1/bps_midi) * start_sec_collection / (1/tempo_bps_collection) )
    midi_start = mido.second2tick(seconds_to_aim_per_midi_tempo, mid.ticks_per_beat, tempo_midi)

    seconds_to_endaim_per_midi_tempo = ((1/bps_midi) * end_sec_collection / (1/tempo_bps_collection) )
    midi_end = mido.second2tick(seconds_to_endaim_per_midi_tempo, mid.ticks_per_beat, tempo_midi)
    
    return midi_start, midi_end

In [26]:
path = '../GeneratedMIDI_webscraping'
store_direct = 'GeneratedMIDISnippets_webscraping'
os.makedirs(store_direct, exist_ok=True)
success_snippeting = []
treated_midi = []
not_in_collection = []

for single_mid_path in glob.glob(path+'/*.mid'):
    
    mid = MidiFile(single_mid_path, clip=True)
    mp3_code_ac = single_mid_path.split('/')[-1].split('.')[0]
    mp3_code = '_'.join(mp3_code_ac.split('_')[:-1])
    midi_start, midi_end = tempo_timeframe_computer(mid, info_df, mp3_code)
    print('MP3_code: ', mp3_code, 'midi start snippet: ', midi_start, 'midi end snippet: ', midi_end)

    # generating time snippet:       
    
    if midi_start != -1 and midi_end != -1: # sample is in the info collection
        pause_collector, nd, note_collector, note_collector_all, SAL, CD, time_passed_ar, nd_short, success = bie.basic_tools_panda(path, mp3_code_ac, midi_start, midi_end)
        success_snippeting.append(success)
        treated_midi.append(mp3_code)
    else:
        # there is no data were the snippet should be cut out. However, according to the project team
        # in this case it the whole piece's mean is taken and clipped for -/+ 15 seconds range. 
        # When we can assume that our midi file is complete, then do that.
        #not_in_collection.append(mp3_code)
        os.makedirs(store_direct, )
        
        shutil.copy2(single_midi_path, store_direct)


MP3_code:  P_Rihanna_Umbrell midi start snippet:  6264.0 midi end snippet:  13780.800000000001
information extraction (and midi snippet generation) done
MP3_code:  K_Tschaikowski_Schwane midi start snippet:  18600.0 midi end snippet:  31000.000000000004
information extraction (and midi snippet generation) done
MP3_code:  P_DanielPowter_BadDay midi start snippet:  9632.0 midi end snippet:  19040.0
information extraction (and midi snippet generation) done
MP3_code:  P_BeachBoys_Wouldnt midi start snippet:  5999.999999999997 midi end snippet:  12749.999999999995
information extraction (and midi snippet generation) done
MP3_code:  K_Bartholdy_Sommern midi start snippet:  310080.0 midi end snippet:  322543.99999999994
information extraction (and midi snippet generation) done
MP3_code:  K_Brahms_Violink midi start snippet:  304560.65480540786 midi end snippet:  349680.75181361637
information extraction (and midi snippet generation) done
MP3_code:  K_Skrjabin_Prelude1 midi start snippet:  218

KeyboardInterrupt: 

For 158 midi files we have an entry in the data collection. For them we try to cut out the relevant midi snippets.
For 33 samples it was not possible to cut out a snippet due to the reason that desired time frame was beyond the time frame of the belonging samples midi:
array(['K_Bartholdy_Sommern', 'P_KingsofLeon_SexOnFi',
       'P_LittleBigTown_GirlCru', 'K_Brahms_Klavier',
       'P_AmyWinehouse_BackToB', 'K_Vivaldi_4JahrWi', 'K_Ravel_Bolero',
       'K_Dvorak_9Sinf4S', 'K_Mozart_DieZaub', 'P_KatyPerry_SwishSw',
       'K_Chopin_EtuedeO', 'K_Beethoven_SinfNr7', 'H_Eminem_TillICo',
       'K_Bartok_PianoSo', 'K_Schumann_Kinders', 'P_DavidGray_PleaseF',
       'K_StraussR_Rosenka', 'K_Vivaldi_4JahrHe', 'K_Mozart_Requiem',
       'K_Mussorgsky_Bildere', 'K_Beethoven_SinfNr6', 'K_Rossini_Wilhelm',
       'K_Bach_Branden', 'K_Strawinsky_Feuervo', 'P_LuisFonsi_Despaci',
       'H_Jay-Z_99Probl', 'P_ChrisBrown_Forever',
       'P_Maggiereilly_Everyti', 'K_Chopin_3KS4Sat', 'K_Vivaldi_4JahrSo',
       'K_Beethoven_SinfNr9', 'P_SamSmith_LayMeDo', 'K_Debussy_Prelude'],
      dtype='<U24').
The files are copied to the snippet files. The belonging files get copied to the location folder of the cut out time snippets.


not_in_collection gives us the midi files for which not entry in the data collection exists. The files can added as whole to the other ones.
['K_Schumann_EtudeS', 'K_Saint-Saens_Carniva', 'H_JayZ_HardKno']. For those MIDI files assume they contain the whole song and take a snippet representing +/- 15 seconds of their mean position when they last more than 30 seconds. Else take the whole song.